In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
config = '../configs/config_shiva.json'
config = json.load(open(config))

In [ ]:
data_dir = config['data_dir']
raw_dir = "{}/{}".format(data_dir, config['raw_data_dir'])
csv_dir = "{}/{}".format(data_dir, config['raw_data_csv'])
ltsf = "{}/ltsf".format(data_dir)

In [ ]:
instrument_file = "{}/{}".format(data_dir, config['stock_meta'])
instrumen_config = json.load(open(instrument_file, "r"))
instrumen_config = {i['instrument_token']: i for i in instrumen_config}

In [ ]:
file = os.listdir(ltsf)[0]
file

In [ ]:
df = pd.read_csv("{}/{}".format(csv_dir, file))

In [ ]:
df.head()

In [ ]:
cols = list(set([i.split('_')[0] for i in df.columns if i != 'date']))

In [ ]:
for c in cols:
    try:
        if instrumen_config[int(c)]['name'].startswith('PC'):
            print(c, instrumen_config[int(c)]['name'])
    except:
        continue

In [ ]:
col_volume_map = {}
for col in cols:   
    if col == 'date':
        continue    
    vol_col = '{}_volume'.format(col)
    avg_col = '{}_avg'.format(col)
    col_volume_map[col] = df[vol_col].mean() * df[avg_col].mean()
col_volume_map = {k: v for k, v in sorted(col_volume_map.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
date_format = "%Y-%m-%d %H:%M:%S"
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
date = df['date'].to_list()
day_list = []
for d in date:
    datetime_obj = datetime.strptime(d, date_format)
    day_of_week = days[datetime_obj.weekday()]
    day_list.append(day_of_week)

In [ ]:
for col in col_volume_map:
         
    fig, ax = plt.subplots()
    fig.set_size_inches(18, 4)
        
    avg_col = '{}_avg'.format(col)
    vol_col = '{}_volume'.format(col)
    volume = df[vol_col].mean()
    
    try:
        name = instrumen_config[int(col)]['name']
    except:
        name = 'unknown'
    title = "Value Traded Mean: {}, Volume mean: {}, Name: {}".format(int(col_volume_map[col]), int(volume), name)
    
    ax.plot(df[avg_col])# ; ax.title(title)# ; plt.show()
    ax.set_title(title, fontsize=18)
    
    for i in range(1, len(day_list)):
        if day_list[i-1] != day_list[i]:
            ax.axvline(x=i, color='k', linestyle='--', linewidth=1)
            ax.text(i, 0.9, day_list[i], transform=ax.get_xaxis_transform(), rotation=90, size=11)
    plt.show()        

In [ ]:
df['2831361_volume'].describe()

In [ ]:
df.head()